<a href="https://colab.research.google.com/github/Priya-Rathor/AIProjects/blob/main/WEAVIATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install weaviate-client
!pip install langchain
!pip install openai

In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [10]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.8/166.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=f04efe3ec0875b00f66f0104058c936479f2f1838226daea0a10be74a969d10a
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [1]:
!pip install "unstructured[pdf]"

In [2]:
OPENAI_API_KEY =""
WEAVIATE_API_KEY ="FsdHwwj9K6OQL6CWnvqPA6qTjoN7cughpQcw"
WEAVIATE_URL = "https://qdonun4bsnozcdeuuuaq.c0.europe-west3.gcp.weaviate.cloud"

# data Reading

In [3]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [4]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader("./data",glob="**/*.pdf")
data =loader.load()


In [5]:
data

[Document(metadata={'source': 'data/Assessment_case_study_2025-01-25 (1).pdf'}, page_content="Assessment Details Report Generated on 1/25/2025\n\nAssessment Type: Case Study\n\nContext: LuxeFashions Pte Ltd, a prominent retail organization based in Singapore, specializes in high-end fashion apparel and accessories. Founded in 2010, LuxeFashions has established itself as a leader in the luxury retail market, boasting a portfolio of both internationally acclaimed brands and exclusive in-house labels. With a strong physical presence through flagship stores located in Singapore's prime shopping districts and an e-commerce platform serving customers across Southeast Asia, LuxeFashions aims to provide an unparalleled shopping experience.\n\nLuxeFashions is focused on expanding its market share and increasing its online sales revenue by 25% over the next fiscal year. The company is also looking to enhance its brand visibility in the region, targeting affluent consumers and fashion enthusiasts

# Text splitting

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 20,
)

docs = text_splitter.split_documents(data)

In [7]:
docs

[Document(metadata={'source': 'data/Assessment_case_study_2025-01-25 (1).pdf'}, page_content="Assessment Details Report Generated on 1/25/2025\n\nAssessment Type: Case Study\n\nContext: LuxeFashions Pte Ltd, a prominent retail organization based in Singapore, specializes in high-end fashion apparel and accessories. Founded in 2010, LuxeFashions has established itself as a leader in the luxury retail market, boasting a portfolio of both internationally acclaimed brands and exclusive in-house labels. With a strong physical presence through flagship stores located in Singapore's prime shopping districts and an e-commerce platform serving customers across Southeast Asia, LuxeFashions aims to provide an unparalleled shopping experience.\n\nLuxeFashions is focused on expanding its market share and increasing its online sales revenue by 25% over the next fiscal year. The company is also looking to enhance its brand visibility in the region, targeting affluent consumers and fashion enthusiasts

In [9]:
len(docs)

14

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key =OPENAI_API_KEY)

# Vector Database Storage

In [ ]:
import weaviate
from langchain.vectorstores import Weaviate
#Connect to weaviate Cluster

auth_config = weaviate.auth.AuthApiKey(api_key = WEAVIATE_API_KEY)
WEAVIATI_URL = WEAVIATE_CLUSTER

client = weaviate.Client(
    url = WEAVIATE_URL,
    additional_headers = {"X-OpenAI-Api-key":OPENAI_API_KEY},
    auth_Client_secret = auth_conjfig,
    startup_period =10
)

In [ ]:
client.is_ready()

In [ ]:


# define input structure
client.schema.delete_all()
client.schema.get()
schema = {
    "classes": [
        {
            "class": "Chatbot",
            "description": "Documents for chatbot",
            "vectorizer": "text2vec-openai",
            "moduleConfig": {"text2vec-openai": {"model": "ada", "type": "text"}},
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-openai": {
                            "skip": False,
                            "vectorizePropertyName": False,
                        }
                    },
                    "name": "content",
                },
            ],
        },
    ]
}
client.schema.create(schema)
vectorstore = Weaviate(client, "Chatbot", "content", attributes=["source"])

In [ ]:
#load text into the vectorstore
text_meta_pair = [(doc.page_content,doc.metadata) for doc in docs]
texts,meta = list(zip(*text_meta_pair))
vectorstore.add_texts(texts,meta)

# similarity measurement

In [ ]:
query = "wha is a yolo?"

# retrieve taxt related to  the query
docs = vectorstore.similarity_search(query,top_k=20)

In [ ]:
docs

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import openAI

In [ ]:
# define  chain

chain = load_qa_chain(
    OpenAI(openai_api_key = OPENAI_API_KEY),
    chain_type = "stuff"
)

In [ ]:
chain.run(input_documents = docs,question = query)